In [1]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import torch
from torchvision import transforms, models
import cv2
import torch
import torchvision
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
model.eval()

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [2]:
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]
# I will link the notebook in the description
# You can copy the class names from the description
# or the notebook
len(COCO_INSTANCE_CATEGORY_NAMES) # 91 classes including background

91

In [3]:
#필요한 함수들 정의
from torchvision import transforms as T
import random
def get_prediction(img_path, threshold=0.5, url=False):
  if url: # We have to request the image
    response = requests.get(img_path)
    img = Image.open(BytesIO(response.content))
  else:
    img = Image.open(img_path) # This is for local images
    
  transform = T.Compose([T.ToTensor()]) # Turn the image into a torch.tensor
  img = transform(img)
  #img = img.cuda() # Only if GPU, otherwise comment this line
  pred = model([img]) # Send the image to the model. This runs on CPU, so its going to take time
  #Let's change it to GPU
  # pred = pred.cpu() # We will just send predictions back to CPU
  # Now we need to extract the bounding boxes and masks
  pred_score = list(pred[0]['scores'].detach().cpu().numpy())
  pred_t = [pred_score.index(x) for x in pred_score if x > threshold][-1]
  masks = (pred[0]['masks'] > 0.5).squeeze().detach().cpu().numpy()
  pred_class = [COCO_INSTANCE_CATEGORY_NAMES[i] for i in list(pred[0]['labels'].cpu().numpy())]
  pred_boxes = [[(i[0], i[1]), (i[2], i[3])] for i in list(pred[0]['boxes'].detach().cpu().numpy())]
  masks = masks[:pred_t+1]
  pred_boxes = pred_boxes[:pred_t+1]
  pred_class = pred_class[:pred_t+1]
  return masks, pred_boxes, pred_class

def random_color_masks(image):
  # I will copy a list of colors here
  colors = [[0, 255, 0],[0, 0, 255],[255, 0, 0],[0, 255, 255],[255, 255, 0],[255, 0, 255],[80, 70, 180], [250, 80, 190],[245, 145, 50],[70, 150, 250],[50, 190, 190]]
  r = np.zeros_like(image).astype(np.uint8)
  g = np.zeros_like(image).astype(np.uint8)
  b = np.zeros_like(image).astype(np.uint8)
  r[image==1], g[image==1], b[image==1] = colors[random.randrange(0, 10)]
  colored_mask = np.stack([r,g,b], axis=2)
  return colored_mask

def instance_segmentation(img_path, threshold=0.5, rect_th=3,
                          text_size=3, text_th=3, url=False):
  masks, boxes, pred_cls = get_prediction(img_path, threshold=threshold, url=url)
  print(pred_cls)
  if url:
    img = url_to_image(img_path) # If we have a url image
  else: # Local image
    img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # For working with RGB images instead of BGR
  for i in range(len(masks)):
    if pred_cls[i] == 'person':
        rgb_mask = random_color_masks(masks[i])
        img = cv2.addWeighted(img, 1, rgb_mask, 0.5, 0)
        mask_list.append(masks[i])
        box_list.append(boxes[i])
  return img, pred_cls, masks[i]

In [5]:
#frame number 바꿔가면서 오류 발생하는지 아닌지 판별
mask_list = []
box_list = []
img, pred_classes, masks = instance_segmentation('./frames(cropped)/frame0.png', rect_th=5, text_th=4)

['person', 'person', 'tie', 'person', 'person']


In [6]:
#frames 폴더 안에 있는 파일들의 파일명을 list로 불러오기
import os
file_list = os.listdir('./frames(cropped)')
len(file_list)
file_list

['frame0.png',
 'frame1.png',
 'frame10.png',
 'frame100.png',
 'frame101.png',
 'frame102.png',
 'frame103.png',
 'frame104.png',
 'frame105.png',
 'frame106.png',
 'frame107.png',
 'frame108.png',
 'frame109.png',
 'frame11.png',
 'frame110.png',
 'frame111.png',
 'frame112.png',
 'frame113.png',
 'frame114.png',
 'frame115.png',
 'frame116.png',
 'frame117.png',
 'frame118.png',
 'frame119.png',
 'frame12.png',
 'frame120.png',
 'frame121.png',
 'frame122.png',
 'frame123.png',
 'frame124.png',
 'frame125.png',
 'frame126.png',
 'frame127.png',
 'frame128.png',
 'frame129.png',
 'frame13.png',
 'frame130.png',
 'frame131.png',
 'frame132.png',
 'frame133.png',
 'frame134.png',
 'frame135.png',
 'frame136.png',
 'frame137.png',
 'frame138.png',
 'frame139.png',
 'frame14.png',
 'frame140.png',
 'frame141.png',
 'frame142.png',
 'frame143.png',
 'frame144.png',
 'frame145.png',
 'frame146.png',
 'frame147.png',
 'frame148.png',
 'frame149.png',
 'frame15.png',
 'frame150.png',
 'frame

In [7]:
list_of_paths =[]
for i in range(890):
    list_of_paths = list_of_paths + ['./frames(cropped)/'+'frame'+str(i)+'.png']
    
list_of_paths

['./frames(cropped)/frame0.png',
 './frames(cropped)/frame1.png',
 './frames(cropped)/frame2.png',
 './frames(cropped)/frame3.png',
 './frames(cropped)/frame4.png',
 './frames(cropped)/frame5.png',
 './frames(cropped)/frame6.png',
 './frames(cropped)/frame7.png',
 './frames(cropped)/frame8.png',
 './frames(cropped)/frame9.png',
 './frames(cropped)/frame10.png',
 './frames(cropped)/frame11.png',
 './frames(cropped)/frame12.png',
 './frames(cropped)/frame13.png',
 './frames(cropped)/frame14.png',
 './frames(cropped)/frame15.png',
 './frames(cropped)/frame16.png',
 './frames(cropped)/frame17.png',
 './frames(cropped)/frame18.png',
 './frames(cropped)/frame19.png',
 './frames(cropped)/frame20.png',
 './frames(cropped)/frame21.png',
 './frames(cropped)/frame22.png',
 './frames(cropped)/frame23.png',
 './frames(cropped)/frame24.png',
 './frames(cropped)/frame25.png',
 './frames(cropped)/frame26.png',
 './frames(cropped)/frame27.png',
 './frames(cropped)/frame28.png',
 './frames(cropped)/fram

In [8]:
number_of_people = [0 for i in range(890)]

In [ ]:
error_numbers = [46, 47, 269, 376, 474, 660, 662, 687, 694, 719, 753, 755, 769, 546, 547, 548, 849]
for i in range(643,890):
 if i in error_numbers:
    continue  
 mask_list = []
 img, pred_classes, masks = instance_segmentation(list_of_paths[i], rect_th=5, text_th=4)
 number_of_people[i] = len(mask_list)
 n = 889-i
 print('현재 frame number:', i)
 print('현재 frame의 사람 수:',len(mask_list))
 print('남은 frame 수:',n)
    
    
    


#7.21
#'axis 2' : 46,47,269,376,474,660,662, 687, 694, 719, 753, 755, 769
#'list index out of range' : 546,547,548, 849번째 이미지 에러 발생

In [10]:
number_of_people = [4, 7, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 4, 5, 5, 4, 3, 3, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2, 0, 2, 2, 2, 4, 4, 2, 2, 2, 2, 2, 2, 6, 4, 3, 1, 1, 0, 0, 1, 1, 2, 3, 3, 1, 1, 2, 2, 1, 1, 1, 1, 3, 5, 3, 4, 5, 4, 4, 5, 4, 2, 2, 3, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 5, 1, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 2, 4, 5, 2, 2, 2, 2, 1, 1, 3, 3, 3, 1, 1, 1, 1, 3, 4, 4, 5, 3, 3, 3, 1, 1, 4, 4, 3, 4, 2, 2, 1, 1, 4, 5, 4, 4, 7, 4, 5, 6, 6, 5, 2, 2, 3, 4, 4, 4, 4, 4, 2, 2, 3, 3, 5, 3, 2, 2, 2, 2, 2, 2, 2, 1, 5, 4, 3, 2, 2, 2, 2, 2, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3, 3, 3, 3, 2, 2, 2, 2, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 5, 2, 5, 3, 4, 5, 4, 4, 4, 4, 3, 3, 2, 2, 4, 4, 4, 4, 4, 5, 5, 5, 6, 5, 4, 2, 2, 3, 4, 1, 1, 13, 13, 14, 12, 11, 2, 1, 1, 0, 2, 3, 3, 1, 1, 4, 4, 3, 2, 2, 1, 1, 0, 1, 1, 1, 3, 2, 1, 1, 2, 2, 2, 2, 2, 4, 1, 1, 1, 1, 1, 3, 3, 3, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 4, 4, 5, 7, 5, 2, 3, 2, 1, 1, 5, 4, 3, 4, 4, 4, 4, 5, 4, 4, 5, 5, 1, 1, 1, 4, 5, 4, 4, 4, 4, 4, 6, 5, 4, 5, 1, 3, 4, 3, 1, 2, 1, 3, 1, 1, 2, 4, 3, 3, 3, 4, 3, 16, 12, 3, 4, 4, 4, 6, 1, 1, 4, 2, 4, 5, 4, 5, 4, 0, 1, 3, 3, 1, 1, 1, 4, 4, 4, 4, 4, 5, 5, 5, 5, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 4, 4, 4, 3, 3, 2, 2, 2, 2, 3, 3, 1, 1, 1, 1, 3, 3, 2, 4, 4, 1, 1, 3, 2, 3, 3, 1, 2, 4, 4, 5, 9, 14, 4, 3, 2, 3, 3, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 3, 2, 3, 3, 3, 3, 1, 1, 1, 1, 2, 2, 3, 3, 3, 3, 3, 3, 4, 2, 2, 2, 2, 0, 1, 2, 3, 3, 2, 5, 5, 3, 4, 1, 1, 1, 2, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 2, 2, 2, 3, 3, 3, 3, 3, 2, 3, 2, 2, 2, 2, 7, 3, 4, 1, 2, 4, 4, 1, 1, 5, 4, 4, 3, 2, 1, 2, 4, 1, 0, 0, 0, 3, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 3, 1, 0, 1, 1, 2, 5, 4, 7, 12, 10, 7, 1, 3, 4, 4, 4, 4, 4, 4, 2, 2, 3, 3, 4, 4, 5, 5, 4, 4, 2, 2, 2, 3, 3, 3, 4, 3, 4, 2, 3, 2, 3, 3, 1, 1, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 5, 3, 3, 3, 3, 2, 2, 1, 1, 1, 4, 4, 4, 3, 1, 1, 1, 1, 4, 2, 2, 2, 2, 2, 2, 2, 1, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 2, 2, 2, 4, 4, 4, 2, 2, 1, 1, 4, 3, 3, 4, 9, 3, 3, 3, 3, 2, 3, 2, 1, 0, 3, 3, 1, 1, 1, 1, 0, 8, 8, 8, 7, 7, 7, 2, 2, 2, 2, 2, 2, 4, 3, 1, 3, 3, 3, 1, 1, 3, 4, 1, 1, 0, 3, 3, 3, 1, 1, 2, 2, 3, 3, 4, 1, 4, 5, 6, 6, 5, 3, 5, 5, 4, 3, 1, 1, 3, 4, 3, 1, 2, 4, 3, 3, 3, 4, 0, 1, 0, 3, 3, 3, 3, 3, 3, 4, 5, 4, 4, 2, 1, 1, 0, 4, 4, 5, 3, 3, 3, 2, 3, 4, 5, 4, 4, 3, 2, 2, 2, 3, 2, 2, 2, 3, 3, 2, 2, 2, 1, 4, 4, 3, 4, 1, 1, 1, 2, 2, 2, 2, 2, 3, 2, 4, 4, 4, 4, 3, 1, 2, 2, 2, 1, 5, 7, 5, 5, 6, 6, 6, 5, 2, 2, 2, 3, 2, 2, 2, 3, 3, 3, 2, 1, 1, 1, 3, 2, 2, 2, 3, 2, 2, 0, 2, 2, 1, 1, 2, 1, 2, 2, 2, 5, 4, 4, 1, 2, 2, 2, 3, 3, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [11]:
print(number_of_people)

[4, 7, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 4, 5, 5, 4, 3, 3, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2, 0, 2, 2, 2, 4, 4, 2, 2, 2, 2, 2, 2, 6, 4, 3, 1, 1, 0, 0, 1, 1, 2, 3, 3, 1, 1, 2, 2, 1, 1, 1, 1, 3, 5, 3, 4, 5, 4, 4, 5, 4, 2, 2, 3, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 5, 1, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 2, 4, 5, 2, 2, 2, 2, 1, 1, 3, 3, 3, 1, 1, 1, 1, 3, 4, 4, 5, 3, 3, 3, 1, 1, 4, 4, 3, 4, 2, 2, 1, 1, 4, 5, 4, 4, 7, 4, 5, 6, 6, 5, 2, 2, 3, 4, 4, 4, 4, 4, 2, 2, 3, 3, 5, 3, 2, 2, 2, 2, 2, 2, 2, 1, 5, 4, 3, 2, 2, 2, 2, 2, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3, 3, 3, 3, 2, 2, 2, 2, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 5, 2, 5, 3, 4, 5, 4, 4, 4, 4, 3, 3, 2, 2, 4, 4, 4, 4, 4, 5, 5, 5, 6, 5, 4, 2, 2, 3, 4, 1, 1, 13, 13, 14, 12, 11, 2, 1, 1, 0, 2, 3, 3, 1, 1, 4, 4, 3, 2, 2, 1, 1, 0, 1, 1, 1, 3, 2, 1, 1, 2, 2, 2, 2, 2, 4, 1, 1, 1, 1, 1, 3, 3, 3, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 4, 4, 5, 7, 5, 2, 3, 2, 1, 1, 5, 4, 3, 4, 4, 4, 4, 5, 4, 4, 5, 5, 1, 1, 1

In [12]:
#평균
a = np.array(number_of_people)
avg = np.mean(a)
print('프레임당 사람 수 평균값:',avg)

프레임당 사람 수 평균값: 2.7797752808988765


In [17]:
#중앙값
import statistics
print('중앙값:', statistics.median(number_of_people))

중앙값: 3.0


In [20]:
#최빈값
from collections import Counter
def most_common(list1):
    
    assert isinstance(list1, list), 'Must be a list type'
    if len(list1) == 0: return None
    return Counter(list1).most_common(n=1)[0][0]
print('최빈값:',most_common(number_of_people))

최빈값: 2
